# Module 3: Large Scale Circuits and Complexity

**Status:** Validated Generator ✅

## 3.1 The Problem: The Exponential Wall

To train an AI model for the AQC hackathon (which uses 100+ qubits), we face a dilemma:
*   To train AI, we need $(Input, Target)$ pairs.
*   To get the Target ($x_{ideal}$), we must simulate the circuit.
*   Simulating 100 qubits requires $2^{100}$ bytes of RAM. **This is more than the atoms in the universe.**

So how do we generate training data?

## 3.2 The Physics Hack: Gottesman-Knill Theorem

There is a special subset of operations called **Clifford Gates** (Hadamard, CNOT, S, Pauli). The Gottesman-Knill theorem proves that any circuit consisting *only* of these gates can be simulated in polynomial time $O(N^2)$.

**Strategy:**
We will generate massive datasets of **Clifford Circuits**. Our AI will learn the noise patterns on these "easy" circuits. Since the *physics of noise* (Relaxation, Crosstalk) is the same for Clifford and Non-Clifford gates, the AI can then generalize to solve "hard" problems.

In [ ]:
import utils  # Shared utilities
from qiskit_aer import AerSimulator
from qiskit import transpile
import time

# --- 1. The Generator (Proof of Speed) ---
# We will define a function to generate thousands of circuits instantly.
def generate_batch(n_samples=1000, n_qubits=10, depth=20):
    print(f"Generating {n_samples} Clifford circuits ({n_qubits} qubits, depth {depth})...")
    
    start_time = time.time()
    
    # We use a Stabilizer simulator which is specialized for Clifford circuits
    sim = AerSimulator(method='stabilizer')
    
    dataset = []
    
    for i in range(n_samples):
        # 1. Generate 
        qc = utils.create_random_clifford_circuit(n_qubits, depth)
        qc.measure_all()
        
        # 2. Transpile & Simulate (Efficiently)
        # Even with 100 qubits, this takes milliseconds because of Gottesman-Knill
        result = sim.run(transpile(qc, sim), shots=1).result()
        
        # In a real pipeline, we'd save the features + result
        dataset.append(qc)
        
        if i % 200 == 0:
            print(f"  Built {i}/{n_samples}...")
            
    end_time = time.time()
    print(f"✅ Done! Generated {n_samples} circuits in {end_time - start_time:.2f} seconds.")
    return dataset

# Run the scale test (Simulating 20 qubits - already harder than a laptop can handle with standard methods)
data = generate_batch(n_samples=1000, n_qubits=20, depth=20)

## 3.3 Topological Constraints (Heavy-Hex)

Real hardware isn't fully connected. IBM processors use a "Heavy Hex" lattice.

**Impact on AI:**
An AI model must know that a CNOT between Qubit 0 and Qubit 1 (Neighbors) is cheap, but a CNOT between Qubit 0 and Qubit 10 (Distant) is expensive because it requires many SWAP gates.

$$ \text{Effective Error} \propto \text{Number of SWAPS} \times \text{Gate Error} $$